In [1]:
!pip install easyAI

  Using cached easyAI-2.0.12-py3-none-any.whl (42 kB)



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
from easyAI import TwoPlayerGame, Human_Player, AI_Player, Negamax
import itertools as it
from game import Dominoes, Board, Deck, Play, Rule


class KingDomino(TwoPlayerGame):

    def __init__(self, players=None):
        # must do these to fit the framework
        self.players = players
        self.nplayer = 1
        self.current_player = 1
        # fill the pile of cards
        self.pile = Deck(Dominoes.from_json("kingdomino.json"), deck_size=40, draw_num=4)
        self.board1 = Board(Rule.MIGHTY_DUEL)
        self.board2 = Board(Rule.MIGHTY_DUEL)
        self.previously_picked_cards1 = [None, None]
        self.previously_picked_cards2 = [None, None]
        self.cards_on_board = self.pile.draw()

    def possible_moves(self):
        if self.nplayer == 1:
            current_board = self.board1
            prev_picked_cards = self.previously_picked_cards1
        else:
            current_board = self.board2
            prev_picked_cards = self.previously_picked_cards2

        # move = choose 2 cards for the next turn and place the previously picked two
        cards_to_pick_next_turn = ["01", "02", "03", "12", "13", "23"]
        
        # for first turn, no previously picked cards
        if prev_picked_cards[0] == None:
            return cards_to_pick_next_turn
            
        positions_for_1st_card = list(current_board.valid_plays(prev_picked_cards[0]))
        positions_for_2nd_card = list(current_board.valid_plays(prev_picked_cards[1]))
        
        positions_to_place = positions_for_1st_card + positions_for_2nd_card
            
        # last turn
        if self.pile.empty():
            #print("LAST TURN, moves:", list(it.product("last turn", positions_to_place)))
            return list(it.product("last turn", positions_to_place))
        
        return list(it.product(cards_to_pick_next_turn, positions_to_place))

    def make_move(self, move):

        if self.nplayer == 1:
            current_board = self.board1
            prev_picked_cards = self.previously_picked_cards1
        else:
            current_board = self.board2
            prev_picked_cards = self.previously_picked_cards2
            print("ciaooooo")

        #print("move", move, "picked", str(prev_picked_cards))
        # if first move
        if type(move) == str:
            prev_picked_cards[0] = self.cards_on_board[int(move[0])]
            prev_picked_cards[1] = self.cards_on_board[int(move[1])]
            return
        
        # if last move
        if move[0] == 'last turn':
            current_board.play(move[1])
        
        cards_picked = move[0]
        prev_picked_cards[0] = self.cards_on_board[int(cards_picked[0])]
        prev_picked_cards[1] = self.cards_on_board[int(cards_picked[1])]
        current_board.play(move[1])
        
        self.cards_on_board = self.pile.draw()

    def is_over(self):
        return self.pile.empty()

    def show(self):
        pass

    def scoring(self):
        if self.nplayer == 1:
            current_board = self.board1
        else:
            current_board = self.board2
        return current_board.points() if self.is_over() else 0

from easyAI import solve_with_iterative_deepening

r,d,m = solve_with_iterative_deepening(
    game=KingDomino(),
    ai_depths=range(1,10),
    win_score=100
)

d:1, a:0, m:01
d:2, a:0, m:01
d:3, a:0, m:01
d:4, a:0, m:01


KeyboardInterrupt: 

In [ ]:
r,d,m

(0, 4, '01')

In [47]:
from easyAI import TwoPlayerGame, Human_Player, AI_Player, Negamax
import itertools as it
from game import Dominoes, Board, Deck, Play, Rule


class KingDomino(TwoPlayerGame):

    def __init__(self, players=None):
        # must do these to fit the framework
        self.players = players
        self.current_player = 1
        # fill the pile of cards
        self.pile = Deck(Dominoes.from_json("kingdomino.json"), deck_size=12, draw_num=4)
        self.board = Board(Rule.MIGHTY_DUEL)
        self.card_to_pick = self.pile.draw()
        self.prev_picked_cards = [None, None]

    def possible_moves(self):

        # move = choose 2 cards for the next turn and place the previously picked two
        if self.pile.empty():
            cards_to_pick_next_turn = ["last turn"]
        else:
            cards_to_pick_next_turn = [(0,1), (0,2), (0,3), (1,2), (1,3), (2,3)]
        
        positions_for_1st_card = list(self.board.valid_plays(self.prev_picked_cards[0]))
        positions_for_2nd_card = list(self.board.valid_plays(self.prev_picked_cards[1]))
        
        positions_to_place = positions_for_1st_card + positions_for_2nd_card
        if positions_to_place == []:
            positions_to_place = ["first turn"]
        
        possible_moves_total = list(it.product(cards_to_pick_next_turn, positions_to_place))
        #print(len(cards_to_pick_next_turn), "pick possibilities", len(positions_to_place), "position to place", len(possible_moves_total), "total")
        return list(it.product(cards_to_pick_next_turn, positions_to_place))

    def make_move(self, move):
        
        if move[0] != "last turn":
            self.prev_picked_cards[0] = self.card_to_pick[move[0][0]]
            self.prev_picked_cards[1] = self.card_to_pick[move[0][1]]
            
        if move[1] != "first turn":
            self.board.play(move[1])
        
        self.card_to_pick = self.pile.draw()

    def is_over(self):
        return self.pile.empty()

    def show(self):
        pass

    def scoring(self):
        return self.board.points() #if self.is_over() else 0

from easyAI import solve_with_iterative_deepening

kd = KingDomino()
r,d,m = solve_with_iterative_deepening(
    game = kd,
    ai_depths = range(1,10),
    win_score = 100,
)

d:1, a:0, m:((0, 1), 'first turn')
d:2, a:0, m:((0, 1), 'first turn')
d:3, a:0, m:((0, 1), 'first turn')
d:4, a:0, m:((0, 1), 'first turn')
d:5, a:0, m:((0, 1), 'first turn')
d:6, a:0, m:((0, 1), 'first turn')
d:7, a:0, m:((0, 1), 'first turn')
d:8, a:0, m:((0, 1), 'first turn')
d:9, a:0, m:((0, 1), 'first turn')


In [44]:
r,d,m

(0, 9, ((0, 1), 'first turn'))

In [45]:
kd.make_move(m)

In [46]:
r,d,m = solve_with_iterative_deepening(
    game = kd,
    ai_depths = range(1,10),
    win_score = 100
)

d:1, a:0, m:((0, 1), 5 5 SOUT)
d:2, a:0, m:((0, 1), 5 5 SOUT)
d:3, a:0, m:((0, 1), 5 5 SOUT)
d:4, a:0, m:((0, 1), 5 5 SOUT)
d:5, a:0, m:((0, 1), 5 5 SOUT)
d:6, a:0, m:((0, 1), 5 5 SOUT)
d:7, a:0, m:((0, 1), 5 5 SOUT)
d:8, a:0, m:((0, 1), 5 5 SOUT)
d:9, a:0, m:((0, 1), 5 5 SOUT)
